In [15]:
# pip install snowflake-connector-python
# pip install snowflake-sqlalchemy
# pip install pyspark findspark

import os
import findspark
findspark.init()
from dotenv import load_dotenv
from pyspark.sql import SparkSession
load_dotenv()

# Initialize a Spark session
spark = (SparkSession.builder
    .appName("Snowflake to PySpark")
    .config("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.2") 
    .master("local[*]")
    .getOrCreate())

# Display the Spark session 
spark

In [16]:
# Snowflake connection options
sfOptions = {
    "sfURL": f"https://{os.getenv('ACCOUNT')}.ap-south-1.aws.snowflakecomputing.com",
    "sfUser": os.getenv('USER_NAME'),
    "sfPassword": os.getenv('PASSWORD'),
    "sfDatabase": "US_ZIP_CODE_CROSSWALK_HUDUPS",
    "sfSchema": "zipcode_crosswalk",
    "sfWarehouse": "compute_wh",
    "sfRole": "ACCOUNTADMIN"
}

# Read the data from Snowflake table into a DataFrame
df = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.DEMOGRAPHICS") \
    .load()

# Register the DataFrame as a temporary SQL view for processing
df.createOrReplaceTempView("demographics")


In [17]:
df.show()

+---------+---------+-----+----------+------------+-----+--------------------+----------------+---------------------+-----------------------+
|ISO3166_1|ISO3166_2| FIPS|  LATITUDE|   LONGITUDE|STATE|              COUNTY|TOTAL_POPULATION|TOTAL_MALE_POPULATION|TOTAL_FEMALE_POPULATION|
+---------+---------+-----+----------+------------+-----+--------------------+----------------+---------------------+-----------------------+
|       US|       AK|02013|55.0952467|-160.0272746|   AK|Aleutians East Bo...|            3299|                 2145|                   1154|
|       US|       AK|02016|51.9594469|  178.338813|   AK|Aleutians West Ce...|            5714|                 3578|                   2136|
|       US|       AK|02020|61.2327493|-148.6669728|   AK|Anchorage Municip...|          299321|               152681|                 146640|
|       US|       AK|02050| 61.350537|-156.4386201|   AK|  Bethel Census Area|           17885|                 9358|                   8527|
|     